# Information Geometry: Mathematical Foundations

## How We Model Information Using Differential Geometry

This notebook explains the **mathematical design** behind the diffgeo library — how differential geometry provides the natural language for describing uncertainty, learning, and inference.

---

## Part 1: The Core Insight — Why Geometry?

### The Problem with Euclidean Thinking

When we optimize a model with parameters $\theta \in \mathbb{R}^n$, standard gradient descent treats parameter space as **flat**:

$$\theta_{t+1} = \theta_t - \eta \nabla_\theta L$$

But this is **coordinate-dependent**! If we reparameterize $\phi = f(\theta)$, the "same" step looks completely different:

$$\|\Delta\theta\|_2 \neq \|\Delta\phi\|_2$$

**Example**: For a Gaussian $\mathcal{N}(\mu, \sigma^2)$, the Euclidean distance from $(\mu, \sigma) = (0, 1)$ to $(0, 2)$ equals the distance to $(0, 101)$ from $(0, 100)$. But statistically, doubling the variance is a massive change; adding 1 to a variance of 100 is negligible.

### The Principle of Covariance

The resolution comes from physics: **physical laws should not depend on coordinate choice**. This is Einstein's *principle of general covariance* [Einstein, 1916].

In machine learning, we adopt an analogous principle:

> **Statistical Covariance Principle**: Quantities that measure statistical relationships should be invariant under reparameterization.

This forces us to use **tensors** — objects with specific transformation rules that cancel coordinate dependence.

### The Geometric Objects

A **smooth manifold** $\mathcal{M}$ is a space that locally looks like $\mathbb{R}^n$ but may have global curvature. The key structures:

| Object | Symbol | Definition | What It Measures |
|--------|--------|------------|------------------|
| **Tangent space** | $T_p\mathcal{M}$ | Vector space of "velocities" at $p$ | Directions you can move |
| **Cotangent space** | $T^*_p\mathcal{M}$ | Dual space of linear functionals | Gradients live here |
| **Metric tensor** | $g_{ij}(p)$ | Smoothly varying inner product | Length, angle, distance |
| **Connection** | $\Gamma^k_{ij}$ | Rule for parallel transport | How to compare vectors at different points |
| **Curvature** | $R^l_{ijk}$ | Failure of parallel transport to close | Intrinsic bending of space |

### The Metric Tensor: Formal Definition

**Definition**: A *Riemannian metric* on $\mathcal{M}$ is a smooth assignment $p \mapsto g_p$ where each $g_p: T_p\mathcal{M} \times T_p\mathcal{M} \to \mathbb{R}$ is:
1. **Symmetric**: $g_p(u, v) = g_p(v, u)$
2. **Positive definite**: $g_p(v, v) > 0$ for $v \neq 0$
3. **Bilinear**: Linear in each argument

In coordinates, $g_p(u, v) = g_{ij}(p) u^i v^j$ (Einstein summation).

The **infinitesimal arc length** is:
$$ds^2 = g_{ij}(\theta) \, d\theta^i \, d\theta^j$$

**Transformation law**: Under $\phi = f(\theta)$:
$$\tilde{g}_{ab}(\phi) = \frac{\partial \theta^i}{\partial \phi^a} \frac{\partial \theta^j}{\partial \phi^b} g_{ij}(\theta)$$

This ensures $ds^2 = g_{ij} d\theta^i d\theta^j = \tilde{g}_{ab} d\phi^a d\phi^b$ — the length element is **invariant**.

### Geodesics: Shortest Paths

**Definition**: A *geodesic* is a curve $\gamma(t)$ that parallel transports its own tangent vector:
$$\nabla_{\dot{\gamma}} \dot{\gamma} = 0$$

In coordinates, this becomes the **geodesic equation**:
$$\ddot{\gamma}^k + \Gamma^k_{ij} \dot{\gamma}^i \dot{\gamma}^j = 0$$

where the **Christoffel symbols** are:
$$\Gamma^k_{ij} = \frac{1}{2} g^{kl} \left( \partial_i g_{jl} + \partial_j g_{il} - \partial_l g_{ij} \right)$$

Geodesics are the "straight lines" of curved space — they minimize arc length locally.

---

### 📚 Further Reading: Part 1

**Foundational texts**:
- **do Carmo, M.P.** (1992). *Riemannian Geometry*. Birkhäuser. — The standard graduate introduction.
- **Lee, J.M.** (2018). *Introduction to Riemannian Manifolds*. Springer. — Modern, rigorous treatment.
- **Spivak, M.** (1979). *A Comprehensive Introduction to Differential Geometry* (5 vols). — Exhaustive, from first principles.

**For machine learning audience**:
- **Absil, P.-A., Mahony, R., Sepulchre, R.** (2008). *Optimization Algorithms on Matrix Manifolds*. Princeton. — Practical focus on optimization.
- **Boumal, N.** (2023). *An Introduction to Optimization on Smooth Manifolds*. Cambridge. — Modern, freely available online.


---

## Part 2: Fisher Information — The Natural Metric on Probability Space

### The Statistical Manifold

Consider a parametric family of probability distributions:
$$\mathcal{S} = \{p(x|\theta) : \theta \in \Theta \subseteq \mathbb{R}^n\}$$

This is a **statistical manifold** — the parameter space $\Theta$ inherits geometric structure from the distributions it parameterizes.

### Definition: Fisher Information Matrix

**Definition** [Fisher, 1925]: The *Fisher Information Matrix* at $\theta$ is:

$$F_{ij}(\theta) = \mathbb{E}_{x \sim p(x|\theta)}\left[\frac{\partial \log p(x|\theta)}{\partial \theta^i} \frac{\partial \log p(x|\theta)}{\partial \theta^j}\right]$$

**Equivalent form** (under regularity conditions):
$$F_{ij}(\theta) = -\mathbb{E}_{x \sim p(x|\theta)}\left[\frac{\partial^2 \log p(x|\theta)}{\partial \theta^i \partial \theta^j}\right]$$

The equivalence follows from:
$$\mathbb{E}\left[\frac{\partial \log p}{\partial \theta^i}\right] = \int \frac{\partial p}{\partial \theta^i} dx = \frac{\partial}{\partial \theta^i} \int p \, dx = \frac{\partial}{\partial \theta^i} (1) = 0$$

### The Cramér-Rao Bound: Why Fisher Information Matters

**Theorem** [Cramér, 1946; Rao, 1945]: For any unbiased estimator $\hat{\theta}$ of $\theta$:

$$\text{Cov}(\hat{\theta}) \geq F(\theta)^{-1}$$

in the sense of positive semidefinite ordering.

**Proof sketch**: Apply Cauchy-Schwarz to the score function $\nabla_\theta \log p(x|\theta)$ and the estimator error $\hat{\theta} - \theta$.

**Interpretation**: The Fisher Information sets a **fundamental limit** on estimation precision. High Fisher = parameters are precisely estimable. Low Fisher = parameters are "sloppy."

### Čencov's Uniqueness Theorem

**Theorem** [Čencov, 1982]: The Fisher metric is the **unique** Riemannian metric on statistical manifolds (up to a constant factor) that satisfies:

1. **Monotonicity under Markov morphisms**: If $\kappa: \mathcal{X} \to \mathcal{Y}$ is a Markov kernel (stochastic map), and $q(y|\theta) = \int \kappa(y|x) p(x|\theta) dx$, then:
   $$F_q(\theta) \leq F_p(\theta)$$
   (Information can only be lost, never gained, by coarse-graining.)

2. **Invariance under sufficient statistics**: If $T(x)$ is sufficient for $\theta$, the Fisher metric computed from $T$ equals that computed from $x$.

**Significance**: This theorem elevates Fisher Information from "a useful quantity" to "the unique correct choice" — it's forced by information-theoretic axioms.

**Proof**: See [Campbell, 1986] for an accessible proof, or [Čencov, 1982] for the original.

### Local Equivalence with KL Divergence

**Theorem**: For nearby distributions:
$$D_{KL}(p_\theta \| p_{\theta + d\theta}) = \frac{1}{2} F_{ij}(\theta) d\theta^i d\theta^j + O(\|d\theta\|^3)$$

**Proof**:
$$D_{KL}(p_\theta \| p_{\theta+d\theta}) = \int p_\theta \log \frac{p_\theta}{p_{\theta+d\theta}} dx$$

Taylor expand $\log p_{\theta+d\theta}$ around $\theta$:
$$\log p_{\theta+d\theta} = \log p_\theta + d\theta^i \partial_i \log p_\theta + \frac{1}{2} d\theta^i d\theta^j \partial_i \partial_j \log p_\theta + O(\|d\theta\|^3)$$

Substituting and using $\mathbb{E}[\partial_i \log p] = 0$:
$$D_{KL} = -\frac{1}{2} d\theta^i d\theta^j \mathbb{E}[\partial_i \partial_j \log p] + O(\|d\theta\|^3) = \frac{1}{2} F_{ij} d\theta^i d\theta^j + O(\|d\theta\|^3)$$

**Interpretation**: The Fisher metric measures the **instantaneous rate of KL divergence** as you move in parameter space.

### Example: Gaussian Family

For the univariate Gaussian $p(x|\mu,\sigma) = \frac{1}{\sigma\sqrt{2\pi}} \exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right)$:

$$\log p = -\log\sigma - \frac{1}{2}\log(2\pi) - \frac{(x-\mu)^2}{2\sigma^2}$$

Computing derivatives and expectations:
$$F_{\mu\mu} = \mathbb{E}\left[\left(\frac{x-\mu}{\sigma^2}\right)^2\right] = \frac{1}{\sigma^2}$$
$$F_{\sigma\sigma} = \mathbb{E}\left[\left(-\frac{1}{\sigma} + \frac{(x-\mu)^2}{\sigma^3}\right)^2\right] = \frac{2}{\sigma^2}$$
$$F_{\mu\sigma} = 0 \quad \text{(by symmetry)}$$

So:
$$F = \begin{pmatrix} 1/\sigma^2 & 0 \\ 0 & 2/\sigma^2 \end{pmatrix}$$

The manifold of Gaussians has **negative curvature** (hyperbolic geometry). The Gaussian family is isometric to the Poincaré half-plane!

### The Score Function and Natural Gradient

The **score function** is $s(\theta; x) = \nabla_\theta \log p(x|\theta)$.

- It has zero mean: $\mathbb{E}[s] = 0$
- Its covariance is Fisher: $\text{Cov}(s) = F$

The **natural gradient** [Amari, 1998] uses Fisher to convert gradients:
$$\tilde{\nabla} L = F^{-1} \nabla L$$

This gives the steepest descent direction in the **intrinsic geometry** of the statistical manifold, not the arbitrary Euclidean geometry of parameter space.

---

### 📚 Further Reading: Part 2

**Original sources**:
- **Fisher, R.A.** (1925). "Theory of Statistical Estimation." *Proc. Cambridge Phil. Soc.* 22:700–725.
- **Rao, C.R.** (1945). "Information and accuracy attainable in estimation." *Bull. Calcutta Math. Soc.* 37:81–91.
- **Čencov, N.N.** (1982). *Statistical Decision Rules and Optimal Inference*. AMS Translations.

**Information geometry**:
- **Amari, S.** (2016). *Information Geometry and Its Applications*. Springer. — The definitive modern treatment by the field's founder.
- **Amari, S. & Nagaoka, H.** (2000). *Methods of Information Geometry*. AMS/Oxford. — More mathematical, covers α-connections.
- **Nielsen, F.** (2020). "An Elementary Introduction to Information Geometry." *Entropy* 22(10):1100. — Accessible overview.

**Natural gradient in ML**:
- **Amari, S.** (1998). "Natural Gradient Works Efficiently in Learning." *Neural Computation* 10(2):251–276.
- **Martens, J.** (2020). "New Insights and Perspectives on the Natural Gradient Method." *JMLR* 21:1–76. — Excellent modern perspective.


In [1]:
# Interactive demo: Fisher metric depends on where you are in parameter space
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

def fisher_gaussian_viz(sigma=1.0):
    """Visualize Fisher metric ellipsoid for Gaussian at different σ values."""
    # Fisher matrix for N(μ, σ²) at given σ
    F = np.array([[1/sigma**2, 0], [0, 2/sigma**2]])
    
    # Create ellipse (unit ball under Fisher metric)
    theta = np.linspace(0, 2*np.pi, 100)
    # Ellipse: F^(-1/2) @ unit circle
    eigvals, eigvecs = np.linalg.eigh(F)
    radii = 1.0 / np.sqrt(eigvals)
    ellipse = eigvecs @ np.diag(radii) @ np.array([np.cos(theta), np.sin(theta)])
    
    fig = make_subplots(rows=1, cols=2, 
                        subplot_titles=[f'Fisher Metric at σ={sigma:.1f}', 
                                       'Gaussian Distribution'],
                        column_widths=[0.5, 0.5])
    
    # Left: Fisher ellipse (unit ball)
    fig.add_trace(go.Scatter(x=ellipse[0], y=ellipse[1], mode='lines',
                            fill='toself', fillcolor='rgba(100,149,237,0.3)',
                            line=dict(color='cornflowerblue', width=2),
                            name='Unit ball'), row=1, col=1)
    fig.add_trace(go.Scatter(x=[0], y=[0], mode='markers',
                            marker=dict(size=10, color='red'),
                            name='Current θ'), row=1, col=1)
    
    # Right: The Gaussian itself
    x = np.linspace(-5, 5, 200)
    y = np.exp(-x**2 / (2*sigma**2)) / (sigma * np.sqrt(2*np.pi))
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', line=dict(width=3, color='coral'),
                            name=f'N(0, {sigma**2:.1f})'), row=1, col=2)
    
    fig.update_xaxes(title_text='Δμ', range=[-3, 3], row=1, col=1)
    fig.update_yaxes(title_text='Δσ', range=[-3, 3], scaleanchor='x', row=1, col=1)
    fig.update_xaxes(title_text='x', row=1, col=2)
    fig.update_yaxes(title_text='p(x)', row=1, col=2)
    
    fig.update_layout(height=400, showlegend=False,
                     title_text='<b>Fisher metric shrinks as σ grows</b><br>' + 
                               '<sub>Small σ → changes are detectable → large metric</sub>')
    return fig

# Interactive slider
sigma_slider = widgets.FloatSlider(value=1.0, min=0.3, max=3.0, step=0.1, 
                                   description='σ:', continuous_update=False)
output = widgets.interactive_output(lambda s: display(fisher_gaussian_viz(s).show()), 
                                   {'s': sigma_slider})
display(sigma_slider, output)


FloatSlider(value=1.0, continuous_update=False, description='σ:', max=3.0, min=0.3)

Output()

---

## Part 3: Vectors vs Covectors — The Fundamental Duality

### The Tangent-Cotangent Duality

At each point $p$ of a manifold $\mathcal{M}$, there are **two** associated vector spaces:

| Space | Name | Elements | Physical Example |
|-------|------|----------|------------------|
| $T_p\mathcal{M}$ | Tangent space | Vectors (velocities) | Velocity of a particle |
| $T^*_p\mathcal{M}$ | Cotangent space | Covectors (1-forms) | Gradient of potential energy |

These are **dual** spaces: $T^*_p\mathcal{M} = (T_p\mathcal{M})^*$.

### Formal Definitions

**Definition**: A *tangent vector* $v \in T_p\mathcal{M}$ is an equivalence class of curves through $p$, or equivalently, a derivation on smooth functions:
$$v: C^\infty(\mathcal{M}) \to \mathbb{R}, \quad v(fg) = v(f)g(p) + f(p)v(g)$$

**Definition**: A *covector* (or *1-form*) $\omega \in T^*_p\mathcal{M}$ is a linear functional:
$$\omega: T_p\mathcal{M} \to \mathbb{R}$$

### Index Notation and Transformation Laws

We use the **Einstein summation convention**: repeated indices (one up, one down) are summed.

**Contravariant vectors** (tangent vectors) have **upper indices**: $v^i$

**Covariant vectors** (covectors) have **lower indices**: $\omega_i$

Under coordinate change $\tilde{\theta}^a = f^a(\theta)$:

**Vectors transform contravariantly**:
$$\tilde{v}^a = \frac{\partial \tilde{\theta}^a}{\partial \theta^i} v^i$$

**Covectors transform covariantly**:
$$\tilde{\omega}_a = \frac{\partial \theta^i}{\partial \tilde{\theta}^a} \omega_i$$

**Key insight**: The product $\omega_i v^i$ is **invariant** — the Jacobians cancel:
$$\tilde{\omega}_a \tilde{v}^a = \frac{\partial \theta^i}{\partial \tilde{\theta}^a} \omega_i \cdot \frac{\partial \tilde{\theta}^a}{\partial \theta^j} v^j = \delta^i_j \omega_i v^j = \omega_i v^i$$

### The Gradient Is a Covector

**Theorem**: For $f: \mathcal{M} \to \mathbb{R}$, the gradient $df$ is naturally a **covector**, not a vector.

**Proof**: The differential $df$ acts on tangent vectors:
$$(df)_p(v) = v(f) = \left.\frac{d}{dt}\right|_{t=0} f(\gamma(t))$$
where $\gamma$ is any curve with $\gamma(0) = p$, $\dot{\gamma}(0) = v$.

In coordinates: $(df)_i = \frac{\partial f}{\partial \theta^i}$ — lower index!

**Why this matters**: When we compute $\nabla_\theta L$ in deep learning, we get a **covector**. But parameter updates require a **vector** (direction to move).

### The Musical Isomorphisms

The metric tensor provides canonical isomorphisms between $T_p\mathcal{M}$ and $T^*_p\mathcal{M}$:

**Flat** (♭): Lower an index (vector → covector)
$$v^\flat_i = g_{ij} v^j$$

**Sharp** (♯): Raise an index (covector → vector)
$$\omega^{\sharp i} = g^{ij} \omega_j$$

These are **inverses**: $(v^\flat)^\sharp = v$ and $(\omega^\sharp)^\flat = \omega$.

### Natural Gradient as Index Raising

Standard gradient descent makes a **type error**:
$$\theta_{t+1} = \theta_t - \eta \nabla_\theta L \quad \text{(WRONG: subtracting covector from point)}$$

Natural gradient **fixes** this by raising the index:
$$\theta_{t+1} = \theta_t - \eta \, (\nabla L)^\sharp = \theta_t - \eta \, F^{-1} \nabla L$$

This is the **geometrically correct** update: convert the gradient covector to a tangent vector using the metric (Fisher Information).

### Why Euclidean Space Hides This

In $\mathbb{R}^n$ with standard basis and Euclidean metric $g_{ij} = \delta_{ij}$:
- $v^\flat_i = \delta_{ij} v^j = v_i$ (components unchanged)
- Vectors and covectors have the same components

This **coincidence** leads to "metric blindness" — confusing objects that are geometrically distinct. On curved manifolds, this confusion causes bugs.

---

### 📚 Further Reading: Part 3

**Mathematical foundations**:
- **Burke, W.L.** (1985). *Applied Differential Geometry*. Cambridge. — Emphasizes the geometric distinction; beautifully written.
- **Frankel, T.** (2011). *The Geometry of Physics*. Cambridge. — Connects to physics; covers differential forms.
- **Schutz, B.** (1980). *Geometrical Methods of Mathematical Physics*. Cambridge. — Excellent index notation pedagogy.

**The gradient as covector**:
- **Baez, J. & Muniain, J.P.** (1994). *Gauge Fields, Knots and Gravity*. World Scientific. Ch. 1 gives beautiful exposition.
- **Misner, Thorne, Wheeler** (1973). *Gravitation*. Freeman. — The "telephone book"; definitive on index gymnastics.


In [2]:
# Interactive demo: Vectors vs Covectors under metric transformation
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

def vector_covector_viz(metric_stretch=1.0, metric_angle=0):
    """Show how vectors and covectors transform differently under a metric."""
    # Metric tensor (stretched ellipse)
    angle_rad = np.radians(metric_angle)
    R = np.array([[np.cos(angle_rad), -np.sin(angle_rad)],
                  [np.sin(angle_rad), np.cos(angle_rad)]])
    D = np.diag([1.0, metric_stretch])
    g = R @ D @ D @ R.T  # g = R D² R^T (SPD matrix)
    g_inv = np.linalg.inv(g)
    
    # A covector (gradient) - fixed
    omega = np.array([1.0, 0.5])
    
    # Convert to vector using metric: v = g^{-1} @ omega
    v = g_inv @ omega
    
    # Create metric ellipse (unit ball)
    theta = np.linspace(0, 2*np.pi, 100)
    eigvals, eigvecs = np.linalg.eigh(g)
    radii = 1.0 / np.sqrt(eigvals)
    ellipse = eigvecs @ np.diag(radii) @ np.array([np.cos(theta), np.sin(theta)])
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=['Covector (gradient) ω', 'Vector v = g⁻¹ω'])
    
    # Left plot: covector as hyperplane level sets
    for c in [-0.5, 0, 0.5, 1.0, 1.5]:
        if abs(omega[1]) > 1e-6:
            x_line = np.linspace(-2, 2, 100)
            y_line = (c - omega[0]*x_line) / omega[1]
            mask = np.abs(y_line) < 2
            fig.add_trace(go.Scatter(x=x_line[mask], y=y_line[mask], mode='lines',
                                    line=dict(color='lightblue', width=1),
                                    showlegend=False), row=1, col=1)
    
    # Covector as arrow (actually represents direction of steepest ascent in Euclidean)
    fig.add_trace(go.Scatter(x=[0, omega[0]], y=[0, omega[1]], mode='lines+markers',
                            line=dict(color='blue', width=3),
                            marker=dict(size=[8, 12], symbol=['circle', 'arrow'],
                                       angleref='previous'),
                            name='Covector ω'), row=1, col=1)
    
    # Right plot: vector and metric ellipse
    fig.add_trace(go.Scatter(x=ellipse[0], y=ellipse[1], mode='lines',
                            fill='toself', fillcolor='rgba(200,200,200,0.3)',
                            line=dict(color='gray', width=2, dash='dash'),
                            name='Metric unit ball'), row=1, col=2)
    fig.add_trace(go.Scatter(x=[0, v[0]], y=[0, v[1]], mode='lines+markers',
                            line=dict(color='red', width=3),
                            marker=dict(size=[8, 12], symbol=['circle', 'arrow'],
                                       angleref='previous'),
                            name='Vector v'), row=1, col=2)
    
    # Show both in right plot for comparison
    fig.add_trace(go.Scatter(x=[0, omega[0]], y=[0, omega[1]], mode='lines',
                            line=dict(color='blue', width=2, dash='dot'),
                            name='Original ω'), row=1, col=2)
    
    for col in [1, 2]:
        fig.update_xaxes(range=[-2.5, 2.5], row=1, col=col)
        fig.update_yaxes(range=[-2.5, 2.5], scaleanchor=f'x{col if col > 1 else ""}', row=1, col=col)
    
    fig.update_layout(height=400, 
                     title_text=f'<b>Metric: stretch={metric_stretch:.1f}, angle={metric_angle}°</b><br>' +
                               '<sub>Blue: covector (gradient). Red: vector (update direction). ' +
                               'They differ when metric ≠ identity!</sub>')
    return fig

# Interactive controls
stretch_slider = widgets.FloatSlider(value=1.0, min=0.3, max=3.0, step=0.1,
                                    description='Stretch:', continuous_update=False)
angle_slider = widgets.IntSlider(value=0, min=-45, max=45, step=5,
                                description='Angle:', continuous_update=False)

def update(stretch, angle):
    display(vector_covector_viz(stretch, angle).show())

out = widgets.interactive_output(update, {'stretch': stretch_slider, 'angle': angle_slider})
display(widgets.HBox([stretch_slider, angle_slider]), out)


Output()

---

## Part 4: The SPD Manifold — Geometry of Covariance Matrices

### The Space $\mathcal{P}_n$ of SPD Matrices

**Definition**: The *symmetric positive definite (SPD) manifold* is:
$$\mathcal{P}_n = \{P \in \mathbb{R}^{n \times n} : P = P^T, \; v^T P v > 0 \; \forall v \neq 0\}$$

**Topology**: $\mathcal{P}_n$ is an **open convex cone** in the vector space $\text{Sym}(n)$ of symmetric matrices:
- Open: Small perturbations preserve positive definiteness
- Convex: $\alpha P + (1-\alpha) Q \in \mathcal{P}_n$ for $P, Q \in \mathcal{P}_n$, $\alpha \in (0,1)$
- Cone: $\lambda P \in \mathcal{P}_n$ for $\lambda > 0$

**Dimension**: $\dim(\mathcal{P}_n) = \dim(\text{Sym}(n)) = \frac{n(n+1)}{2}$

### The Tangent Space

At $P \in \mathcal{P}_n$, the tangent space is the space of symmetric matrices:
$$T_P \mathcal{P}_n \cong \text{Sym}(n) = \{V \in \mathbb{R}^{n \times n} : V = V^T\}$$

### The Affine-Invariant (AI) Metric

**Definition** [Pennec et al., 2006]: The *affine-invariant Riemannian metric* at $P \in \mathcal{P}_n$:

$$\langle V, W \rangle_P = \text{tr}(P^{-1} V P^{-1} W)$$

**Theorem** [Skovgaard, 1984]: This metric has the following properties:

1. **GL(n)-invariance**: For any invertible $A \in GL(n)$:
   $$d(APA^T, AQA^T) = d(P, Q)$$
   
2. **Inversion invariance**: $d(P^{-1}, Q^{-1}) = d(P, Q)$

3. **Geodesic completeness**: Geodesics extend for all time

4. **Non-positive sectional curvature**: $\mathcal{P}_n$ is a **Hadamard manifold** (simply connected, non-positive curvature)

**Why GL(n)-invariance matters**: If we change coordinates by $\tilde{x} = Ax$, covariances transform as $\tilde{\Sigma} = A\Sigma A^T$. The AI metric is **blind to this change** — the geometry is intrinsic.

### Geodesics: Closed-Form Solution

**Theorem**: The geodesic from $P$ to $Q$ is:
$$\gamma(t) = P^{1/2} \left(P^{-1/2} Q P^{-1/2}\right)^t P^{1/2}$$

where $M^t$ for SPD $M$ means: diagonalize $M = U \Lambda U^T$, then $M^t = U \Lambda^t U^T$.

**Proof sketch**: Verify that $\gamma$ satisfies the geodesic equation $\nabla_{\dot{\gamma}} \dot{\gamma} = 0$ where $\nabla$ is the Levi-Civita connection of the AI metric.

**Special cases**:
- $\gamma(0) = P$
- $\gamma(1) = Q$
- $\gamma(1/2)$ = **geometric mean** of $P$ and $Q$

### The Geodesic Distance

**Theorem**: The Riemannian distance is:
$$d(P, Q) = \left\| \log(P^{-1/2} Q P^{-1/2}) \right\|_F = \sqrt{\sum_{i=1}^n \log^2 \lambda_i}$$

where $\lambda_1, \ldots, \lambda_n$ are the eigenvalues of $P^{-1}Q$ (equivalently, the generalized eigenvalues of $(Q, P)$).

**Properties**:
- $d(P, Q) = 0 \Leftrightarrow P = Q$
- $d(P, Q) = d(Q, P)$ (symmetry)
- $d(P, R) \leq d(P, Q) + d(Q, R)$ (triangle inequality)

### The Swelling Effect: Why Euclidean is Wrong

**Theorem** [Arsigny et al., 2007]: The Euclidean mean of SPD matrices **swells**:
$$\det\left(\frac{1}{N}\sum_{i=1}^N P_i\right) \geq \left(\prod_{i=1}^N \det(P_i)\right)^{1/N}$$

with equality iff all $P_i$ are equal.

**The Fréchet mean** (Riemannian center of mass) avoids this:
$$\bar{P} = \arg\min_{P \in \mathcal{P}_n} \sum_{i=1}^N d(P, P_i)^2$$

For $N=2$: $\bar{P} = \gamma(1/2)$ = geometric mean.

### Connection to Fisher Information

**Theorem** [Skovgaard, 1984]: For the centered Gaussian family $\mathcal{N}(0, \Sigma)$, the Fisher Information metric on $\Sigma$ equals the affine-invariant metric on $\mathcal{P}_n$ (up to a factor of 2).

**Proof**: For $p(x|\Sigma) = \frac{1}{(2\pi)^{n/2}|\Sigma|^{1/2}} \exp(-\frac{1}{2}x^T\Sigma^{-1}x)$:

$$\log p = -\frac{n}{2}\log(2\pi) - \frac{1}{2}\log|\Sigma| - \frac{1}{2}x^T\Sigma^{-1}x$$

The Fisher metric works out to:
$$F(V, W) = \frac{1}{2}\text{tr}(\Sigma^{-1} V \Sigma^{-1} W)$$

This is exactly (1/2 times) the AI metric!

**Significance**: The "natural" Riemannian geometry of $\mathcal{P}_n$ and the "natural" information geometry of Gaussian covariances are **the same**. This is not coincidence — it reflects deep structure.

### Other Metrics on $\mathcal{P}_n$

| Metric | Formula | Properties |
|--------|---------|------------|
| **Affine-Invariant** | $\langle V,W\rangle_P = \text{tr}(P^{-1}VP^{-1}W)$ | GL(n)-invariant, geodesically complete |
| **Log-Euclidean** | $d(P,Q) = \|\log P - \log Q\|_F$ | Commutative, fast to compute |
| **Bures-Wasserstein** | $d(P,Q)^2 = \text{tr}(P) + \text{tr}(Q) - 2\text{tr}(P^{1/2}QP^{1/2})^{1/2}$ | Optimal transport metric |
| **Power-Euclidean** | $d_\alpha(P,Q) = \|P^\alpha - Q^\alpha\|_F$ | Interpolates AI (α→0) and Euclidean (α=1) |

---

### 📚 Further Reading: Part 4

**Foundational papers**:
- **Pennec, X., Fillard, P., Ayache, N.** (2006). "A Riemannian Framework for Tensor Computing." *IJCV* 66(1):41–66. — The foundational paper for DTI and BCI applications.
- **Skovgaard, L.T.** (1984). "A Riemannian Geometry of the Multivariate Normal Model." *Scand. J. Stat.* 11(4):211–223.
- **Arsigny, V. et al.** (2007). "Geometric Means in a Novel Vector Space Structure on SPD Matrices." *SIAM J. Matrix Anal.* 29(1):328–347.

**Applications**:
- **Barachant, A. et al.** (2012). "Multiclass Brain-Computer Interface Classification by Riemannian Geometry." *IEEE Trans. Biomed. Eng.* 59(4):920–928. — BCI classification using SPD geometry.
- **Yger, F. et al.** (2017). "Riemannian Approaches in Brain-Computer Interfaces: A Review." *IEEE Trans. NSRE* 25(10):1753–1762.

**Textbooks**:
- **Bhatia, R.** (2007). *Positive Definite Matrices*. Princeton. — Comprehensive mathematical treatment.
- **Bridson, M. & Haefliger, A.** (1999). *Metric Spaces of Non-Positive Curvature*. Springer. — For Hadamard manifold theory.


In [3]:
# Interactive demo: SPD Geodesics and the "swelling effect"
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

def spd_geodesic_viz(t=0.5, show_swelling=True):
    """Visualize geodesic interpolation on SPD manifold vs Euclidean."""
    # Two 2x2 SPD matrices (displayed as ellipses)
    P = np.array([[2.0, 0.5], [0.5, 1.0]])
    Q = np.array([[1.0, -0.3], [-0.3, 2.5]])
    
    # Euclidean interpolation (WRONG for SPD)
    M_euclidean = (1-t) * P + t * Q
    
    # Riemannian geodesic (CORRECT)
    P_sqrt = np.linalg.cholesky(P)
    P_sqrt_inv = np.linalg.inv(P_sqrt)
    M = P_sqrt_inv @ Q @ P_sqrt_inv.T
    eigvals, eigvecs = np.linalg.eigh(M)
    M_t = eigvecs @ np.diag(eigvals**t) @ eigvecs.T
    M_riemannian = P_sqrt @ M_t @ P_sqrt.T
    
    def make_ellipse(matrix, n_points=100):
        theta = np.linspace(0, 2*np.pi, n_points)
        circle = np.array([np.cos(theta), np.sin(theta)])
        L = np.linalg.cholesky(matrix)
        return L @ circle
    
    fig = make_subplots(rows=1, cols=2, 
                       subplot_titles=['Euclidean Interpolation (Wrong)', 
                                      'Riemannian Geodesic (Correct)'])
    
    colors = {'P': 'blue', 'Q': 'red', 'interp': 'green'}
    
    for col, (M_interp, title) in enumerate([(M_euclidean, 'Euclidean'), 
                                              (M_riemannian, 'Riemannian')], 1):
        # Start ellipse
        e_P = make_ellipse(P)
        fig.add_trace(go.Scatter(x=e_P[0], y=e_P[1], mode='lines',
                                line=dict(color=colors['P'], width=2),
                                name='P (start)' if col==1 else None,
                                showlegend=(col==1)), row=1, col=col)
        
        # End ellipse
        e_Q = make_ellipse(Q)
        fig.add_trace(go.Scatter(x=e_Q[0], y=e_Q[1], mode='lines',
                                line=dict(color=colors['Q'], width=2),
                                name='Q (end)' if col==1 else None,
                                showlegend=(col==1)), row=1, col=col)
        
        # Interpolated ellipse
        e_M = make_ellipse(M_interp)
        fig.add_trace(go.Scatter(x=e_M[0], y=e_M[1], mode='lines',
                                line=dict(color=colors['interp'], width=3),
                                fill='toself', fillcolor='rgba(0,255,0,0.2)',
                                name=f'Interpolated (t={t:.1f})' if col==1 else None,
                                showlegend=(col==1)), row=1, col=col)
    
    # Show determinant (measure of "size") 
    det_P, det_Q = np.linalg.det(P), np.linalg.det(Q)
    det_euc, det_riem = np.linalg.det(M_euclidean), np.linalg.det(M_riemannian)
    
    for col in [1, 2]:
        fig.update_xaxes(range=[-3, 3], row=1, col=col)
        fig.update_yaxes(range=[-3, 3], scaleanchor=f'x{col if col > 1 else ""}', row=1, col=col)
    
    swelling_text = (f'<br><sub>det(P)={det_P:.2f}, det(Q)={det_Q:.2f} | '
                    f'Euclidean det={det_euc:.2f}, Riemannian det={det_riem:.2f}</sub>')
    
    fig.update_layout(height=450, 
                     title_text=f'<b>SPD Geodesic at t={t:.1f}</b>' + swelling_text +
                               '<br><sub>Euclidean interpolation "swells" (det increases), '
                               'Riemannian stays geometrically consistent</sub>')
    return fig

# Interactive slider
t_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05,
                              description='t:', continuous_update=False)
out = widgets.interactive_output(lambda t: display(spd_geodesic_viz(t).show()), {'t': t_slider})
display(t_slider, out)


FloatSlider(value=0.5, continuous_update=False, description='t:', max=1.0, step=0.05)

Output()

---

## Part 5: Finsler Geometry — Beyond Symmetric Distances

### Motivation: Asymmetry in Nature and Statistics

Riemannian geometry assumes **symmetric** distances: $d(p, q) = d(q, p)$.

But asymmetry is ubiquitous:
- **KL divergence**: $D_{KL}(P \| Q) \neq D_{KL}(Q \| P)$
- **Directed graphs**: Edge $A \to B$ doesn't imply $B \to A$
- **Thermodynamics**: Entropy increases; time has an arrow
- **Biology**: Predator-prey dynamics, infection spread
- **Economics**: Transaction costs differ by direction

### Finsler Manifolds: Formal Definition

**Definition** [Finsler, 1918]: A *Finsler structure* on manifold $M$ is a function $F: TM \to [0, \infty)$ such that:

1. **Smoothness**: $F$ is $C^\infty$ on $TM \setminus \{0\}$

2. **Positive homogeneity**: $F(x, \lambda v) = \lambda F(x, v)$ for $\lambda > 0$

3. **Strong convexity**: The **fundamental tensor**
   $$g_{ij}(x, v) = \frac{1}{2} \frac{\partial^2 F^2}{\partial v^i \partial v^j}$$
   is positive definite for $v \neq 0$

**Key difference from Riemannian**: The metric $g_{ij}$ depends on **both position and direction**!

### The Indicatrix (Unit Ball)

**Definition**: The *indicatrix* at $x$ is:
$$\Sigma_x = \{v \in T_x M : F(x, v) = 1\}$$

- **Riemannian case**: $\Sigma_x$ is an ellipsoid centered at origin
- **Finsler case**: $\Sigma_x$ can be any smooth, strongly convex body

**Physical interpretation**: $\Sigma_x$ is the set of velocities you can achieve in unit time from $x$.

### The Randers Metric

**Definition** [Randers, 1941]: The *Randers metric* is:
$$F(x, v) = \alpha(x, v) + \beta(x, v)$$

where:
- $\alpha(x, v) = \sqrt{a_{ij}(x) v^i v^j}$ is a Riemannian norm
- $\beta(x, v) = b_i(x) v^i$ is a 1-form (drift)

**Explicit form**:
$$F(v) = \sqrt{v^T A v} + b^T v$$

**Validity condition**: $\|b\|_\alpha = \sqrt{b^T A^{-1} b} < 1$ (otherwise $F$ can be negative)

### Geometric Interpretation: Zermelo Navigation

**Theorem** [Zermelo, 1931]: A Randers metric models **navigation in a wind field**.

- $\alpha$: Your walking/swimming speed (isotropic)
- $\beta$: Ambient drift (wind, current)
- $F(v)$: Effective cost to move in direction $v$

**Going with the wind**: $F(v)$ is small (cheap)
**Going against the wind**: $F(-v)$ is large (expensive)

**Asymmetry ratio**:
$$\frac{F(v)}{F(-v)} = \frac{\alpha + \beta}{\alpha - \beta}$$

### The Fundamental Tensor

For Randers metric $F = \alpha + \beta$, the fundamental tensor is:

$$g_{ij}(v) = \frac{\alpha}{F} a_{ij} + \frac{\beta}{F} \frac{\partial \alpha}{\partial v^i} \frac{\partial \alpha}{\partial v^j} - \frac{\alpha}{F^2} \left( \frac{\partial \alpha}{\partial v^i} b_j + \frac{\partial \alpha}{\partial v^j} b_i \right) + \frac{b_i b_j}{F}$$

**Key point**: $g_{ij}$ depends on $v$, unlike Riemannian metrics.

### Geodesics in Finsler Geometry

Finsler geodesics minimize arc length:
$$\text{Length}(\gamma) = \int_0^1 F(\gamma(t), \dot{\gamma}(t)) \, dt$$

The geodesic equation involves the **Chern connection** (or Cartan connection):
$$\ddot{\gamma}^k + 2 G^k(\gamma, \dot{\gamma}) = 0$$

where $G^k$ are the **geodesic spray coefficients**.

**For Randers**: Geodesics are related to Riemannian geodesics via a "navigation" transformation [Robles, 2007].

### Connection to Information Geometry

**Theorem** [Shen, 2003]: The KL divergence can be locally approximated as:
$$D_{KL}(p_\theta \| p_{\theta + v}) = \frac{1}{2} g_{ij}(\theta) v^i v^j + \frac{1}{6} T_{ijk}(\theta) v^i v^j v^k + O(\|v\|^4)$$

where:
- $g_{ij}$ = Fisher metric (symmetric, Riemannian)
- $T_{ijk}$ = **skewness tensor** (captures asymmetry)

**The cubic term** $T_{ijk}$ is what makes KL asymmetric. To capture this faithfully requires Finsler geometry.

### Why Finsler for Machine Learning?

1. **Directed distances**: Model causal or irreversible relationships
2. **Asymmetric costs**: Different costs for moving forward vs. backward in parameter space
3. **Information asymmetry**: Better model for KL-based objectives
4. **Physical constraints**: Navigation problems with drift

---

### 📚 Further Reading: Part 5

**Foundational texts**:
- **Bao, D., Chern, S.-S., Shen, Z.** (2000). *An Introduction to Riemann-Finsler Geometry*. Springer. — The definitive graduate text.
- **Shen, Z.** (2001). *Differential Geometry of Spray and Finsler Spaces*. Kluwer.
- **Randers, G.** (1941). "On an Asymmetrical Metric in the Four-Space of General Relativity." *Phys. Rev.* 59:195–199.

**Navigation and Zermelo**:
- **Zermelo, E.** (1931). "Über das Navigationsproblem bei ruhender oder veränderlicher Windverteilung." *ZAMM* 11:114–124.
- **Robles, C.** (2007). "Geodesics in Randers Spaces of Constant Curvature." *Trans. AMS* 359(4):1633–1651.

**Information-theoretic connections**:
- **Shen, Z.** (2003). "Finsler Metrics with K = 0 and S = 0." *Canadian J. Math.* 55(1):112–132.
- **Ohta, S.** (2009). "Finsler Interpolation Inequalities." *Calc. Var.* 36:211–249.

**Applications**:
- **Astola, L. & Florack, L.** (2011). "Finsler Geometry on Higher Order Tensor Fields and Applications to High Angular Resolution Diffusion Imaging." *IJCV* 92:325–336. — Finsler in medical imaging.


In [4]:
# Interactive demo: Finsler (Randers) metric asymmetry
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

def randers_viz(drift_x=0.3, drift_y=0.0):
    """Visualize Randers indicatrix (unit ball) showing asymmetry."""
    A = np.eye(2)  # Riemannian part = identity
    b = np.array([drift_x, drift_y])
    
    # Check validity: ||b||_A < 1
    b_norm = np.sqrt(b @ np.linalg.solve(A, b))
    if b_norm >= 0.99:
        b = b * 0.95 / b_norm  # Clamp to valid range
    
    # Randers norm: F(v) = sqrt(v'Av) + b'v
    def randers_norm(v):
        return np.sqrt(v @ A @ v) + b @ v
    
    # Sample unit vectors and compute F(v) for each
    theta = np.linspace(0, 2*np.pi, 200)
    directions = np.array([np.cos(theta), np.sin(theta)]).T
    
    # Randers indicatrix: F(v) = 1 surface
    # For each direction, find radius r such that F(r*d) = 1
    indicatrix = []
    for d in directions:
        # F(r*d) = r*sqrt(d'Ad) + r*(b'd) = r*(alpha + beta) = 1
        # where alpha = ||d||_A, beta = b'd
        alpha = np.sqrt(d @ A @ d)
        beta = b @ d
        if alpha + beta > 0:
            r = 1.0 / (alpha + beta)
            indicatrix.append(r * d)
        else:
            indicatrix.append([np.nan, np.nan])
    indicatrix = np.array(indicatrix)
    
    # Riemannian unit ball (for comparison)
    riemannian_ball = np.array([np.cos(theta), np.sin(theta)]).T
    
    fig = make_subplots(rows=1, cols=2, 
                       subplot_titles=['Indicatrix (Unit Ball)', 
                                      'Distance Asymmetry: d(O,P) vs d(P,O)'])
    
    # Left: indicatrix
    fig.add_trace(go.Scatter(x=riemannian_ball[:,0], y=riemannian_ball[:,1], 
                            mode='lines', line=dict(color='gray', dash='dash', width=2),
                            name='Riemannian (symmetric)'), row=1, col=1)
    fig.add_trace(go.Scatter(x=indicatrix[:,0], y=indicatrix[:,1],
                            mode='lines', line=dict(color='crimson', width=3),
                            fill='toself', fillcolor='rgba(220,20,60,0.2)',
                            name='Randers (asymmetric)'), row=1, col=1)
    fig.add_trace(go.Scatter(x=[0, b[0]*2], y=[0, b[1]*2], mode='lines+markers',
                            line=dict(color='blue', width=2),
                            marker=dict(size=[8, 12], symbol=['circle', 'arrow-up'],
                                       angleref='previous'),
                            name='Drift vector b'), row=1, col=1)
    fig.add_trace(go.Scatter(x=[0], y=[0], mode='markers',
                            marker=dict(size=10, color='black'), 
                            name='Origin'), row=1, col=1)
    
    # Right: distance comparison heatmap
    # Show d(0, P) - d(P, 0) for points P on a grid
    grid_size = 30
    x_grid = np.linspace(-1.5, 1.5, grid_size)
    y_grid = np.linspace(-1.5, 1.5, grid_size)
    X, Y = np.meshgrid(x_grid, y_grid)
    
    asymmetry = np.zeros_like(X)
    for i in range(grid_size):
        for j in range(grid_size):
            P = np.array([X[i,j], Y[i,j]])
            if np.linalg.norm(P) > 0.01:
                d_forward = randers_norm(P)  # d(0, P) ≈ F(P) for small P
                d_backward = randers_norm(-P)  # d(P, 0) ≈ F(-P)
                asymmetry[i,j] = (d_forward - d_backward) / (d_forward + d_backward + 1e-8)
            else:
                asymmetry[i,j] = 0
    
    fig.add_trace(go.Heatmap(x=x_grid, y=y_grid, z=asymmetry,
                            colorscale='RdBu', zmid=0,
                            colorbar=dict(title='Asymmetry', x=1.02),
                            name='Asymmetry'), row=1, col=2)
    fig.add_trace(go.Scatter(x=[0, b[0]*2], y=[0, b[1]*2], mode='lines+markers',
                            line=dict(color='black', width=2),
                            marker=dict(size=[6, 10]),
                            showlegend=False), row=1, col=2)
    
    for col in [1, 2]:
        fig.update_xaxes(range=[-2, 2], row=1, col=col)
        fig.update_yaxes(range=[-2, 2], scaleanchor=f'x{col if col > 1 else ""}', row=1, col=col)
    
    fig.update_layout(height=450,
                     title_text=f'<b>Randers Metric: b=({drift_x:.2f}, {drift_y:.2f})</b><br>' +
                               '<sub>Going "with the wind" (drift direction) is cheaper than against it</sub>')
    return fig

# Interactive controls
dx_slider = widgets.FloatSlider(value=0.3, min=-0.8, max=0.8, step=0.05,
                               description='Drift x:', continuous_update=False)
dy_slider = widgets.FloatSlider(value=0.0, min=-0.8, max=0.8, step=0.05,
                               description='Drift y:', continuous_update=False)

def update(dx, dy):
    display(randers_viz(dx, dy).show())

out = widgets.interactive_output(update, {'dx': dx_slider, 'dy': dy_slider})
display(widgets.HBox([dx_slider, dy_slider]), out)


Output()

---

## Part 6: Dually Flat Manifolds and Bregman Divergences

### Beyond Metric: The α-Connection Family

A Riemannian manifold has a unique torsion-free, metric-compatible connection (Levi-Civita). But statistical manifolds have a **family** of natural connections.

**Definition** [Amari, 1985]: The *α-connection* $\nabla^{(\alpha)}$ for $\alpha \in \mathbb{R}$ is defined by:
$$\Gamma^{(\alpha)}_{ij,k} = \mathbb{E}\left[\left(\partial_i \partial_j \ell + \frac{1-\alpha}{2} \partial_i \ell \, \partial_j \ell\right) \partial_k \ell\right]$$

where $\ell = \log p(x|\theta)$.

**Special cases**:
- $\alpha = 0$: Levi-Civita connection (metric-compatible)
- $\alpha = 1$: **e-connection** $\nabla^{(e)}$ (exponential)
- $\alpha = -1$: **m-connection** $\nabla^{(m)}$ (mixture)

### Duality: The α and -α Connections

**Theorem** [Amari, 1985]: The $\nabla^{(\alpha)}$ and $\nabla^{(-\alpha)}$ connections are **dually coupled**:
$$X \cdot g(Y, Z) = g(\nabla^{(\alpha)}_X Y, Z) + g(Y, \nabla^{(-\alpha)}_X Z)$$

This means: if you parallel transport along $\nabla^{(\alpha)}$ in one slot, you must use $\nabla^{(-\alpha)}$ in the other to preserve inner products.

### Exponential Families: The Canonical Example

**Definition**: An *exponential family* has the form:
$$p(x|\theta) = h(x) \exp\left(\theta^i T_i(x) - \psi(\theta)\right)$$

where:
- $\theta = (\theta^1, \ldots, \theta^n)$: **natural parameters**
- $T(x) = (T_1(x), \ldots, T_n(x))$: **sufficient statistics**
- $\psi(\theta) = \log \int h(x) \exp(\theta \cdot T(x)) dx$: **log-partition function** (cumulant generating function)

### The Two Coordinate Systems

**Natural coordinates** $\theta$: The parameters in the exponential form.

**Expectation coordinates** $\eta$: The expected sufficient statistics:
$$\eta_i = \mathbb{E}_{p_\theta}[T_i(x)] = \frac{\partial \psi}{\partial \theta^i}$$

**Legendre duality**: The map $\theta \mapsto \eta = \nabla\psi(\theta)$ is a diffeomorphism (assuming minimal exponential family). The inverse is $\eta \mapsto \theta = \nabla\phi(\eta)$ where $\phi(\eta) = \theta \cdot \eta - \psi(\theta)$ is the **conjugate potential**.

### Dually Flat Structure

**Theorem** [Amari & Nagaoka, 2000]: On an exponential family:

1. The e-connection $\nabla^{(e)}$ is **flat in $\theta$ coordinates**: Christoffel symbols vanish
2. The m-connection $\nabla^{(m)}$ is **flat in $\eta$ coordinates**: Christoffel symbols vanish
3. The Fisher metric has the form:
   $$g_{ij}(\theta) = \frac{\partial^2 \psi}{\partial \theta^i \partial \theta^j} = \frac{\partial \eta_i}{\partial \theta^j}$$

**Geodesics**:
- e-geodesics: Straight lines in $\theta$ coordinates
- m-geodesics: Straight lines in $\eta$ coordinates

### Bregman Divergences

**Definition** [Bregman, 1967]: For strictly convex $\phi: \Omega \to \mathbb{R}$:
$$D_\phi(p \| q) = \phi(p) - \phi(q) - \langle \nabla\phi(q), p - q \rangle$$

**Properties**:
- $D_\phi(p \| q) \geq 0$ (by convexity)
- $D_\phi(p \| q) = 0 \Leftrightarrow p = q$
- Generally **asymmetric**: $D_\phi(p \| q) \neq D_\phi(q \| p)$

### The KL-Bregman Connection

**Theorem**: For exponential families:
$$D_{KL}(p_\theta \| p_{\theta'}) = D_\psi(\theta' \| \theta) = D_\phi(\eta \| \eta')$$

where $D_\psi$ uses the log-partition function, and $D_\phi$ uses the conjugate potential.

**Proof**: Direct calculation using $D_{KL} = \mathbb{E}_\theta[\log p_\theta - \log p_{\theta'}]$ and the exponential family form.

### Pythagorean Theorem

**Theorem** [Amari, 1985]: If $r$ lies on the e-geodesic from $p$ to $q$, then:
$$D_{KL}(p \| q) = D_{KL}(p \| r) + D_{KL}(r \| q)$$

Similarly for m-geodesics with arguments reversed.

**Consequence**: Bregman projections onto e-flat or m-flat submanifolds satisfy a generalized Pythagorean relation.

### Applications in Machine Learning

| Algorithm | Geometric Interpretation |
|-----------|-------------------------|
| **EM algorithm** | Alternating e- and m-projections |
| **Natural gradient** | Following e-geodesics |
| **Mirror descent** | Following m-geodesics |
| **Information projection** | m-projection onto constraint set |
| **Maximum likelihood** | m-projection of empirical distribution |
| **Maximum entropy** | e-projection onto moment constraints |

---

### 📚 Further Reading: Part 6

**Original sources**:
- **Amari, S.** (1985). *Differential-Geometrical Methods in Statistics*. Springer Lecture Notes in Statistics 28.
- **Bregman, L.M.** (1967). "The Relaxation Method of Finding the Common Point of Convex Sets." *USSR Computational Math. & Math. Physics* 7(3):200–217.

**Comprehensive treatments**:
- **Amari, S. & Nagaoka, H.** (2000). *Methods of Information Geometry*. AMS/Oxford. — The definitive mathematical treatment of dually flat structures.
- **Nielsen, F. & Nock, R.** (2009). "Sided and Symmetrized Bregman Centroids." *IEEE Trans. IT* 55(6):2882–2904.

**Machine learning connections**:
- **Banerjee, A. et al.** (2005). "Clustering with Bregman Divergences." *JMLR* 6:1705–1749. — Bregman divergences unify many clustering algorithms.
- **Raskutti, G. & Mukherjee, S.** (2015). "The Information Geometry of Mirror Descent." *IEEE Trans. IT* 61(3):1451–1457.


---

## Part 7: The Sloppy Model Phenomenon

### The Universality of Sloppiness

**Empirical observation** [Sethna et al., 2014]: In diverse complex models — systems biology, neural networks, materials science — the Fisher Information eigenvalues span **many orders of magnitude**:

$$\lambda_1 \gg \lambda_2 \gg \cdots \gg \lambda_n$$

Often 6–10 orders of magnitude, with eigenvalues roughly evenly spaced on a log scale!

### The Model Manifold

**Definition**: The *model manifold* is the image of parameter space in prediction space:
$$\mathcal{M} = \{y(\theta) : \theta \in \Theta\} \subset \mathbb{R}^m$$

where $y(\theta)$ is the model's prediction at parameters $\theta$.

The Fisher Information $F_{ij} = \sum_k \frac{\partial y_k}{\partial \theta^i} \frac{\partial y_k}{\partial \theta^j}$ is the **metric tensor** of $\mathcal{M}$ embedded in prediction space.

### The Hyper-Ribbon Geometry

**Theorem** (informal) [Transtrum & Qiu, 2014]: Sloppy model manifolds have the geometry of a **hyper-ribbon**:
- Bounded extent in most directions (width $\sim O(1)$)
- Exponentially varying widths along principal axes
- Effectively low-dimensional despite high parameter count

**Eigenvalue spectrum**: For sloppy models:
$$\lambda_k \approx \lambda_0 \cdot \exp(-k/k_0)$$

The eigenvalues decay exponentially — this is **not** a power law but steeper.

### Stiff vs Sloppy Parameter Combinations

| Property | Stiff Directions | Sloppy Directions |
|----------|------------------|-------------------|
| Eigenvalue | Large $\lambda$ | Small $\lambda$ |
| Effect on predictions | Large | Negligible |
| Constrained by data | Strongly | Weakly |
| Uncertainty | Small | Large |
| Identifiability | Well-identified | Poorly identified |

### The Cramér-Rao Interpretation

Recall the Cramér-Rao bound: $\text{Cov}(\hat{\theta}) \geq F^{-1}$.

- **Stiff directions**: Large $\lambda$ → small uncertainty → well-estimated
- **Sloppy directions**: Small $\lambda$ → large uncertainty → poorly estimated

**Key insight**: Sloppy directions are **statistically irrelevant** — the data doesn't constrain them, so we don't need to estimate them precisely.

### Effective Dimensionality

**Definition**: The *effective dimensionality* measures how many directions matter:
$$d_{\text{eff}} = \frac{(\text{tr } F)^2}{\text{tr}(F^2)} = \frac{(\sum_i \lambda_i)^2}{\sum_i \lambda_i^2}$$

For uniform eigenvalues: $d_{\text{eff}} = n$.
For sloppy models: $d_{\text{eff}} \ll n$.

**Alternative**: Count eigenvalues above a threshold (e.g., 1% of maximum).

### Why Overparameterization Works

**Paradox**: Neural networks have millions of parameters but generalize well. Classical theory says they should overfit.

**Sloppy resolution**: Most parameter directions are sloppy:
- They don't affect predictions much
- The model is effectively low-dimensional
- "Degrees of freedom" ≈ $d_{\text{eff}}$ ≪ parameter count

**Connection to double descent**: The transition from classical to modern ML regime occurs when stiff directions saturate.

### Implications for Optimization

**Natural gradient** adapts to sloppiness automatically:
$$\Delta\theta = -F^{-1} \nabla L$$

- **Stiff directions** (large $\lambda$): $F^{-1}$ is small → small steps (already constrained)
- **Sloppy directions** (small $\lambda$): $F^{-1}$ is large → large steps (need exploration)

This is exactly the right behavior — focus on directions that matter!

### Geometric Insight: Curvature and Information

The eigenvalues of Fisher = principal curvatures of the model manifold.

- **High curvature** (stiff): Small changes in parameters → large changes in predictions
- **Low curvature** (sloppy): Large changes in parameters → predictions barely change

The geometry **reveals the structure** of the learning problem.

### Connection to Our Data

In EEG covariance matrices from brain-computer interfaces:

| Direction Type | Interpretation |
|----------------|----------------|
| **Stiff** | Channel combinations that reliably distinguish brain states (left vs right motor imagery) |
| **Sloppy** | Channel combinations that vary randomly trial-to-trial (noise, artifacts) |

The eigenvalue spectrum of the data covariance **reveals which features are informative** for classification — a geometric feature selection!

---

### 📚 Further Reading: Part 7

**Foundational papers**:
- **Brown, K.S. & Sethna, J.P.** (2003). "Statistical Mechanical Approaches to Models with Many Poorly Known Parameters." *Phys. Rev. E* 68:021904.
- **Transtrum, M.K. et al.** (2010). "Perspective: Sloppiness and Emergent Theories in Physics, Biology, and Beyond." *J. Chem. Phys.* 143:010901.
- **Machta, B.B. et al.** (2013). "Parameter Space Compression Underlies Emergent Theories and Predictive Models." *Science* 342:604–607.

**Geometry of sloppiness**:
- **Transtrum, M.K. & Qiu, P.** (2014). "Model Reduction by Manifold Boundaries." *Phys. Rev. Lett.* 113:098701.
- **Quinn, K.N. et al.** (2019). "Visualizing Probabilistic Models in Minkowski Space." *arXiv:1905.05072*.

**Connections to deep learning**:
- **Karakida, R. et al.** (2019). "Universal Statistics of Fisher Information in Deep Neural Networks." *NeurIPS*.
- **Fort, S. et al.** (2019). "Emergent Properties of the Local Geometry of Neural Loss Landscapes." *arXiv:1910.05929*.
- **Papyan, V.** (2020). "Traces of Class/Cross-Class Structure Pervade Deep Learning Spectra." *JMLR* 21:1–64.


In [5]:
# Interactive demo: Sloppy model eigenvalue spectrum
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

def sloppy_model_viz(n_params=20, stiffness_ratio=4.0):
    """Visualize the 'sloppy model' eigenvalue spectrum phenomenon."""
    # Generate a Fisher-like matrix with exponentially decaying eigenvalues
    # This mimics real sloppy models
    eigenvalues = np.array([10**(stiffness_ratio * (1 - i/(n_params-1))) for i in range(n_params)])
    
    # Create random orthogonal matrix for eigenvectors
    np.random.seed(42)  # Reproducible
    Q, _ = np.linalg.qr(np.random.randn(n_params, n_params))
    fisher = Q @ np.diag(eigenvalues) @ Q.T
    
    fig = make_subplots(rows=1, cols=2,
                       subplot_titles=['Eigenvalue Spectrum (log scale)',
                                      'Model Manifold Geometry (2D projection)'])
    
    # Left: eigenvalue spectrum
    fig.add_trace(go.Scatter(x=list(range(1, n_params+1)), y=eigenvalues,
                            mode='lines+markers',
                            line=dict(color='steelblue', width=2),
                            marker=dict(size=8),
                            name='Eigenvalues'), row=1, col=1)
    
    # Mark stiff vs sloppy threshold (e.g., 1% of max)
    threshold = eigenvalues[0] * 0.01
    n_stiff = np.sum(eigenvalues > threshold)
    fig.add_hline(y=threshold, line_dash='dash', line_color='red', row=1, col=1)
    fig.add_annotation(x=n_params/2, y=threshold*2, text=f'{n_stiff} stiff, {n_params-n_stiff} sloppy',
                      showarrow=False, row=1, col=1)
    
    fig.update_yaxes(type='log', title='Eigenvalue', row=1, col=1)
    fig.update_xaxes(title='Parameter index', row=1, col=1)
    
    # Right: 2D projection showing hyper-ribbon shape
    # Sample points on a "model manifold" with sloppy structure
    n_samples = 500
    coords = np.random.randn(n_samples, n_params)
    # Scale by inverse sqrt of eigenvalues (sloppy directions extend further)
    coords = coords / np.sqrt(eigenvalues)
    
    # Project to 2D using first two eigenvectors
    proj_stiff = coords @ Q[:, 0]  # Stiff direction
    proj_sloppy = coords @ Q[:, -1]  # Most sloppy direction
    
    fig.add_trace(go.Scatter(x=proj_sloppy, y=proj_stiff,
                            mode='markers',
                            marker=dict(size=4, color='coral', opacity=0.5),
                            name='Model predictions'), row=1, col=2)
    
    fig.update_xaxes(title='Sloppy direction', row=1, col=2)
    fig.update_yaxes(title='Stiff direction', row=1, col=2)
    
    span = np.log10(eigenvalues[0] / eigenvalues[-1])
    fig.update_layout(height=450,
                     title_text=f'<b>Sloppy Model: {n_params} parameters, {span:.1f} orders of magnitude</b><br>' +
                               '<sub>The model manifold is a "hyper-ribbon" — flat in sloppy directions, curved in stiff</sub>')
    return fig

# Interactive controls
n_params_slider = widgets.IntSlider(value=20, min=5, max=50, step=5,
                                   description='# params:', continuous_update=False)
ratio_slider = widgets.FloatSlider(value=4.0, min=1.0, max=8.0, step=0.5,
                                  description='Log span:', continuous_update=False)

def update(n, r):
    display(sloppy_model_viz(n, r).show())

out = widgets.interactive_output(update, {'n': n_params_slider, 'r': ratio_slider})
display(widgets.HBox([n_params_slider, ratio_slider]), out)


Output()

---

## Part 8: Library Design — How the Pieces Fit Together

### The Conceptual Architecture

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                           RAW DATA                                          │
│         (time series, images, graphs, covariance matrices)                  │
└─────────────────────────────────────────────────────────────────────────────┘
                                    │
                                    ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                     GEOMETRY EXTRACTION                                     │
│                                                                             │
│   DataGeometryExtractor: Fit probabilistic model → Fisher metric emerges   │
│   SPDGeometryExtractor: Covariance matrices → Affine-invariant metric      │
└─────────────────────────────────────────────────────────────────────────────┘
                                    │
                                    ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                     GEOMETRIC STRUCTURES                                    │
│                                                                             │
│   ┌──────────────┐    ┌──────────────┐    ┌──────────────┐                 │
│   │ MetricTensor │    │ SPDManifold  │    │ RandersMetric│                 │
│   │ (Riemannian) │    │ (Covariance) │    │ (Finsler)    │                 │
│   └──────────────┘    └──────────────┘    └──────────────┘                 │
│           │                   │                   │                         │
│           └───────────────────┴───────────────────┘                         │
│                               │                                             │
│                    ┌──────────┴──────────┐                                  │
│                    │   FisherMetric      │                                  │
│                    │   (Information)     │                                  │
│                    └─────────────────────┘                                  │
└─────────────────────────────────────────────────────────────────────────────┘
                                    │
                                    ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                     GEOMETRIC OPERATIONS                                    │
│                                                                             │
│   • Geodesic computation (shortest paths)                                   │
│   • Parallel transport (moving vectors along curves)                        │
│   • Fréchet mean (geometric average)                                        │
│   • Natural gradient (geometry-aware optimization)                          │
│   • Dualization (vector ↔ covector conversion)                              │
└─────────────────────────────────────────────────────────────────────────────┘
                                    │
                                    ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                     APPLICATIONS                                            │
│                                                                             │
│   • Classification (Riemannian classifiers on SPD)                          │
│   • Optimization (natural gradient descent)                                 │
│   • Interpolation (geodesic interpolation)                                  │
│   • Embedding (preserve geometric structure)                                │
└─────────────────────────────────────────────────────────────────────────────┘
```

### Key Design Decisions

| Decision | Rationale |
|----------|-----------|
| **Explicit tensor variance tracking** | Prevents mixing vectors/covectors (a common source of coordinate-dependent bugs) |
| **Metric as first-class object** | All geometric operations flow from the metric; changing metric changes behavior consistently |
| **Fisher = metric** | Fisher Information IS a Riemannian metric; we don't treat them as separate concepts |
| **Finsler extension** | Real data often has asymmetry; Randers metrics handle this gracefully |
| **Geometry extraction** | Users shouldn't need to hand-craft manifolds; learn geometry from data |


---

## Summary: The Geometric View of Information

### The Core Ideas

| # | Concept | Mathematical Statement | Practical Implication |
|---|---------|----------------------|----------------------|
| 1 | **Statistical manifold** | Distributions form manifold $\mathcal{S}$ | Parameter spaces have intrinsic geometry |
| 2 | **Fisher metric** | Unique metric satisfying Čencov's axioms | Use $F^{-1}\nabla L$ not $\nabla L$ for updates |
| 3 | **Vector ≠ Covector** | $\nabla L \in T^*$, updates $\in T$ | Metric converts between them |
| 4 | **SPD geometry** | $\mathcal{P}_n$ with AI metric is Hadamard | Use geodesic mean, not arithmetic mean |
| 5 | **Finsler for asymmetry** | $F(v) \neq F(-v)$ allowed | Model KL divergence, directed graphs |
| 6 | **Dual flatness** | $\theta$ and $\eta$ coordinates both flat | Bregman projections, EM algorithm |
| 7 | **Sloppiness** | Eigenvalues span orders of magnitude | Effective dimension ≪ parameter count |

### The Unifying Theme: Covariance

**All of these are aspects of COVARIANCE in the geometric sense:**

| Aspect | How It Manifests |
|--------|-----------------|
| **Tensor covariance** | Objects transform consistently under coordinate changes |
| **Statistical covariance** | Fisher metric measures parameter co-variation with predictions |
| **Random variable covariance** | SPD manifold is the space of covariance matrices |
| **Directional covariance** | Finsler metrics capture how forward/backward costs co-vary |

### The Mathematical Chain

$$\text{Data} \xrightarrow{\text{model}} p(x|\theta) \xrightarrow{\text{Fisher}} g_{ij}(\theta) \xrightarrow{\text{geometry}} \text{geodesics, means, gradients}$$

Each arrow preserves structure when done correctly:
- Model fitting respects data geometry
- Fisher metric emerges from the model
- Geometric operations respect the metric

### Key Theorems Summary

1. **Čencov (1982)**: Fisher is the unique information-monotonic metric
2. **Cramér-Rao (1945)**: $\text{Cov}(\hat\theta) \geq F^{-1}$ — Fisher bounds estimation
3. **Amari (1985)**: Exponential families are dually flat
4. **Skovgaard (1984)**: Gaussian Fisher = SPD affine-invariant metric
5. **Zermelo (1931)**: Randers metrics model navigation with drift

---

## 📚 Consolidated Bibliography

### Foundational Texts

**Differential Geometry**:
- do Carmo, M.P. (1992). *Riemannian Geometry*. Birkhäuser.
- Lee, J.M. (2018). *Introduction to Riemannian Manifolds*. Springer.
- Burke, W.L. (1985). *Applied Differential Geometry*. Cambridge.

**Information Geometry**:
- Amari, S. (2016). *Information Geometry and Its Applications*. Springer.
- Amari, S. & Nagaoka, H. (2000). *Methods of Information Geometry*. AMS/Oxford.
- Nielsen, F. (2020). "An Elementary Introduction to Information Geometry." *Entropy* 22(10):1100.

**SPD Manifolds**:
- Bhatia, R. (2007). *Positive Definite Matrices*. Princeton.
- Pennec, X. et al. (2006). "A Riemannian Framework for Tensor Computing." *IJCV* 66(1):41–66.

**Finsler Geometry**:
- Bao, D., Chern, S.-S., Shen, Z. (2000). *An Introduction to Riemann-Finsler Geometry*. Springer.

### Key Papers

- Fisher, R.A. (1925). "Theory of Statistical Estimation." *Proc. Cambridge Phil. Soc.* 22:700–725.
- Rao, C.R. (1945). "Information and accuracy attainable in estimation." *Bull. Calcutta Math. Soc.* 37:81–91.
- Čencov, N.N. (1982). *Statistical Decision Rules and Optimal Inference*. AMS.
- Amari, S. (1998). "Natural Gradient Works Efficiently in Learning." *Neural Computation* 10(2):251–276.
- Martens, J. (2020). "New Insights on the Natural Gradient Method." *JMLR* 21:1–76.
- Transtrum, M.K. et al. (2015). "Sloppiness and Emergent Theories." *J. Chem. Phys.* 143:010901.

### Applications

- Barachant, A. et al. (2012). "Multiclass BCI Classification by Riemannian Geometry." *IEEE Trans. BME* 59(4):920–928.
- Banerjee, A. et al. (2005). "Clustering with Bregman Divergences." *JMLR* 6:1705–1749.
- Absil, P.-A. et al. (2008). *Optimization Algorithms on Matrix Manifolds*. Princeton.


---

## Appendix: Seeing It In Practice

Let's see these concepts with actual data from the library.


In [6]:
import sys
sys.path.insert(0, '..')

import numpy as np

# Load EEG data as SPD covariance matrices
from data.loaders import PhysioNetEEGLoader

loader = PhysioNetEEGLoader()
dataset = loader.load_subject("S001", runs=[3, 4])  # Left vs right fist

print(f"Loaded {dataset.n_samples} covariance matrices")
print(f"Each matrix is {dataset.manifold_dim}×{dataset.manifold_dim}")
print(f"Labels: {np.unique(dataset.labels)} (1=left fist, 2=right fist)")


Loaded 496 covariance matrices
Each matrix is 64×64
Labels: [1 2] (1=left fist, 2=right fist)


In [7]:
# The SPD Manifold in action
from diffgeo.geometry import SPDManifold

# Create manifold
dim = dataset.manifold_dim
manifold = SPDManifold(dim)

# Take two covariance matrices
P = dataset.matrices[0]  
Q = dataset.matrices[1]

# Euclidean distance (WRONG for covariances)
d_euclidean = np.linalg.norm(P - Q, 'fro')

# Riemannian distance (CORRECT)
d_riemannian = manifold.distance(P, Q)

print(f"Euclidean distance: {d_euclidean:.4f}")
print(f"Riemannian distance: {d_riemannian:.4f}")
print(f"\nRatio: {d_riemannian/d_euclidean:.2f}x")
print("\nThe Riemannian distance accounts for the curved geometry of SPD space!")


rocm_plugin_extension not found


Euclidean distance: 6.8211
Riemannian distance: 11.2735

Ratio: 1.65x

The Riemannian distance accounts for the curved geometry of SPD space!


In [9]:
# Data geometry reveals stiff/sloppy directions
# For raw data, the sample covariance captures the same geometric structure
# (For a Gaussian model, Fisher = inverse covariance!)

# Flatten covariance matrices to vectors and compute their sample covariance
subset_matrices = dataset.matrices[:50, :8, :8]  # First 50 samples, 8×8 submatrix
data_vectors = subset_matrices.reshape(50, -1)  # Each matrix → 64-dim vector

# Sample covariance of the data (this IS the data geometry)
data_covariance = np.cov(data_vectors, rowvar=False)

# Analyze eigenvalue spectrum (precision matrix = Fisher for Gaussian)
eigenvalues = np.linalg.eigvalsh(data_covariance)
eigenvalues = eigenvalues[::-1]  # Descending order
eigenvalues = eigenvalues[eigenvalues > 1e-10]  # Remove numerical zeros

print("Data covariance eigenvalue spectrum:")
print(f"  Largest:  {eigenvalues[0]:.2e}")
print(f"  Smallest: {eigenvalues[-1]:.2e}")
print(f"  Condition number: {eigenvalues[0]/eigenvalues[-1]:.2e}")
print(f"\nSpan: {np.log10(eigenvalues[0]/eigenvalues[-1]):.1f} orders of magnitude")
print(f"\nEffective dimensionality: ~{np.sum(eigenvalues > eigenvalues[0]*0.01)} of {len(eigenvalues)} dimensions")
print("\nThis is the 'sloppy model' phenomenon — most directions are noise!")


Data covariance eigenvalue spectrum:
  Largest:  1.68e+00
  Smallest: 7.34e-07
  Condition number: 2.29e+06

Span: 6.4 orders of magnitude

Effective dimensionality: ~5 of 36 dimensions

This is the 'sloppy model' phenomenon — most directions are noise!


In [10]:
# Finsler geometry for asymmetric distances
from diffgeo.geometry import RandersMetric

# Create a Randers metric (Riemannian + drift)
A = np.eye(3)  # Riemannian part
b = np.array([0.3, 0.1, 0.0])  # Drift vector

randers = RandersMetric(A, b)

# Test asymmetry
v = np.array([1.0, 0.0, 0.0])  # Direction vector

forward_cost = randers.norm(v)
backward_cost = randers.norm(-v)

print(f"Randers metric asymmetry demo:")
print(f"  Forward  (with drift):    F(v)  = {forward_cost:.4f}")
print(f"  Backward (against drift): F(-v) = {backward_cost:.4f}")
print(f"  Ratio: {backward_cost/forward_cost:.2f}x")
print(f"\nGoing against the 'wind' costs {(backward_cost/forward_cost - 1)*100:.0f}% more!")
print("\nThis models directed relationships, causal effects, irreversibility...")


Randers metric asymmetry demo:
  Forward  (with drift):    F(v)  = 1.3000
  Backward (against drift): F(-v) = 0.7000
  Ratio: 0.54x

Going against the 'wind' costs -46% more!

This models directed relationships, causal effects, irreversibility...
